In [14]:
import pandas as pd
import numpy as np
import scipy.io
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from tensorflow import keras
from tensorflow.keras import layers
from datasetResize import *


In [15]:
# lecture du excel
labels_df = pd.read_csv('REFERENCE-V3.csv', header=None)
labels_df.columns = ['filename', 'label']

print(labels_df.head())

  filename label
0   A00001     N
1   A00002     N
2   A00003     N
3   A00004     A
4   A00005     A


In [33]:
# Charger tous les signaux et leurs labels
labels = []
signals = []
signals_padded = []
signals_truncated = []
signals_interpolated = []

# Initialiser avec les paramètres pour le redimensionnement
ecg_resizer_max = ECGResizing(target_length=18286)
ecg_resizer_med = ECGResizing(target_length=9000)

for index, row in labels_df.iterrows():
    
    labels.append(row['label'])
    filename = row['filename']
    # Charger un signal ECG
    signal = ecg_resizer_max.load_ecg(f'training2017/{filename}.mat')
    #signals = np.append(signals,[[signal]])
    signals.append(signal)

    # Appliquer le zero padding
    signal_padded = ecg_resizer_max.resize_signal(signal, method='padding')
    signals_padded.append(signal_padded)

    # Appliquer le tronquage
    signal_truncated = ecg_resizer_med.resize_signal(signal, method='padding')
    signals_truncated.append(signal_truncated)

    # Appliquer l'interpolation
    signal_interpolated = ecg_resizer_med.resize_signal(signal, method='interpolate')
    signals_interpolated.append(signal_interpolated)

    if index%10 == 0:
        print("len sig = ",index, len(signals))


len sig =  0 1
len sig =  10 11
len sig =  20 21
len sig =  30 31
len sig =  40 41
len sig =  50 51
len sig =  60 61
len sig =  70 71
len sig =  80 81
len sig =  90 91
len sig =  100 101
len sig =  110 111
len sig =  120 121
len sig =  130 131
len sig =  140 141
len sig =  150 151
len sig =  160 161
len sig =  170 171
len sig =  180 181
len sig =  190 191
len sig =  200 201
len sig =  210 211
len sig =  220 221
len sig =  230 231
len sig =  240 241
len sig =  250 251
len sig =  260 261
len sig =  270 271
len sig =  280 281
len sig =  290 291
len sig =  300 301
len sig =  310 311
len sig =  320 321
len sig =  330 331
len sig =  340 341
len sig =  350 351
len sig =  360 361
len sig =  370 371
len sig =  380 381
len sig =  390 391
len sig =  400 401
len sig =  410 411
len sig =  420 421
len sig =  430 431
len sig =  440 441
len sig =  450 451
len sig =  460 461
len sig =  470 471
len sig =  480 481
len sig =  490 491
len sig =  500 501
len sig =  510 511
len sig =  520 521
len sig =  530 

In [38]:
print(len(signals[0]))
print(len(signals_padded[0]))
print(len(signals_truncated[0]))
print(len(signals_interpolated[0]))

9000
18286
9000
9000


In [40]:
labels = np.array(labels)
signals_padded = np.array(signals_padded)
signals_truncated = np.array(signals_truncated)
signals_interpolated = np.array(signals_interpolated)


# Encoder les labels
label_mapping = {'N': 0, 'A': 1, 'O': 2, '~': 3}
labels_encoded = np.vectorize(label_mapping.get)(labels)


In [41]:
# Diviser en ensembles d'entraînement et de test
#X_train_signals, X_test_signals, y_train_signals, y_test_signals = train_test_split(signals, labels_encoded, test_size=0.2, stratify=labels, random_state=42)
X_train_padded, X_test_padded, y_train_padded, y_test_padded = train_test_split(signals_padded, labels_encoded, test_size=0.2, stratify=labels, random_state=42)
X_train_truncated, X_test_truncated, y_train_truncated, y_test_truncated = train_test_split(signals_truncated, labels_encoded, test_size=0.2, stratify=labels, random_state=42)
X_train_interpolated, X_test_interpolated, y_train_interpolated, y_test_interpolated = train_test_split(signals_interpolated, labels_encoded, test_size=0.2, stratify=labels, random_state=42)

# Afficher les formes des ensembles
print(f'Ensemble d\'entraînement : {X_train_padded.shape}, Ensemble de test : {X_test_padded.shape}')

Ensemble d'entraînement : (6822, 18286), Ensemble de test : (1706, 18286)
